In [ ]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 googleads

In [2]:
import json
import os
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scopes required by the Google Ads API
SCOPES = ['https://www.googleapis.com/auth/adwords']

# Load client secrets JSON file
CLIENT_SECRETS_FILE = 'client_secrets.json'

# Define the token file where the access and refresh tokens will be stored
TOKEN_FILE = 'token.json'

# Function to handle OAuth2 flow
def get_credentials():
    creds = None

    # Check if token already exists
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If no valid credentials, start the OAuth2 flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for future use
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

    return creds

# Get credentials
credentials = get_credentials()

# Now you can use these credentials to make requests to the Google Ads API

In [ ]:
!pip install google-ads

In [5]:
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException
import os

# Set the path to your google-ads.yaml file
os.environ['GOOGLE_ADS_CONFIGURATION_FILE_PATH'] = 'google-ads.yaml'

# Create a GoogleAdsClient instance
client = GoogleAdsClient.load_from_storage()

In [ ]:
# dismisses ALL recommendations that can be dismissed through the API (almost all)

# Updated CIDs as requested
customer_ids = [
    '0000000000', '0000000000', '0000000000', '0000000000', '0000000000'
]

# Query to fetch recommendations
query = """
    SELECT
        recommendation.resource_name,
        recommendation.type
    FROM recommendation
    WHERE recommendation.dismissed = false
"""

# Initialize services
ga_service = client.get_service("GoogleAdsService")
recommendation_service = client.get_service("RecommendationService")

# Loop through each customer ID
for customer_id in customer_ids:
    print(f"\nProcessing Customer ID: {customer_id}")
    
    try:
        # Fetch recommendations
        response = ga_service.search_stream(customer_id=customer_id, query=query)
        
        dismissed_count = 0
        has_recommendations = False
        
        # Process and dismiss recommendations
        for batch in response:
            has_recommendations = True
            for row in batch.results:
                try:
                    # Create dismiss operation
                    dismiss_operation = client.get_type("DismissRecommendationRequest")
                    dismiss_operation.customer_id = customer_id
                    dismiss_operation.operations = [{
                        "resource_name": row.recommendation.resource_name
                    }]
                    
                    # Execute dismiss
                    result = recommendation_service.dismiss_recommendation(
                        request=dismiss_operation
                    )
                    dismissed_count += 1
                except Exception as e:
                    print(f"Error dismissing recommendation: {e}")
        
        if not has_recommendations:
            print("No recommendations found to dismiss")
        else:
            print(f"Total recommendations dismissed: {dismissed_count}")
            
    except Exception as e:
        print(f"Error processing account: {e}")